In [1]:
import pandas as pd
import numpy as np

In [2]:
# Import SQL Alchemy
import sqlalchemy
from sqlalchemy import create_engine, MetaData

# Import and establish Base for which classes will be constructed 
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

In [3]:
# Import modules to declare columns and column data types
from sqlalchemy import Column, Integer, String, Float, Text, Numeric

from sqlalchemy import inspect

In [4]:
#csv to df
access_logs_df = pd.read_csv('/home/spinster/bootcamp/homework/apache/infiles/cleaned_access_logs.csv')
responses_df = pd.read_csv('../infiles/Hitesh_IP_demographics.csv')


In [5]:
access_logs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30694 entries, 0 to 30693
Data columns (total 10 columns):
Unnamed: 0      30694 non-null int64
ip              30694 non-null object
DateTime        30694 non-null object
TimeZone        30694 non-null object
Method          30694 non-null object
URL             30694 non-null object
ResponseCode    30694 non-null object
BytesSent       30694 non-null object
Referer         30694 non-null object
UserAgent       30694 non-null object
dtypes: int64(1), object(9)
memory usage: 2.3+ MB


In [6]:
responses_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7729 entries, 0 to 7728
Data columns (total 19 columns):
city                            6024 non-null object
connection_isp                  7523 non-null object
continent_code                  3951 non-null object
continent_name                  7724 non-null object
country_code                    7703 non-null object
country_name                    7704 non-null object
currency_code                   7704 non-null object
ip                              7729 non-null object
latitude                        7724 non-null float64
location_capital                7697 non-null object
longitude                       7724 non-null float64
region_name                     6239 non-null object
time_zone_code                  7331 non-null object
time_zone_current_time          7331 non-null object
time_zone_gmt_offset            7325 non-null float64
time_zone_id                    7331 non-null object
time_zone_is_daylight_saving    7331 non-n

In [7]:
#drop things not used
responses_df = responses_df.drop(['continent_code', 'country_code', 'currency_code', 'location_capital', 'time_zone_code','time_zone_current_time','time_zone_id','time_zone_is_daylight_saving','type','zip'], axis=1)

In [21]:
responses_df

,city,connection_isp,continent_name,country_name,ip,latitude,longitude,region_name,time_zone_gmt_offset
0,Priolo Gargallo,Linkem spa,Europe,Italy,62.170.2.205,37.1500,15.1833,Sicily,7200.0
1,Brooklyn,"MCI Communications Services, Inc. d/b/a Verizo...",North America,United States,108.41.39.187,40.5833,-73.9240,New York,-14400.0
2,Seattle,Wowrack.com,North America,United States,216.244.66.194,47.4891,-122.2908,Washington,-25200.0
3,Guangzhou,China Telecom (Group),Asia,China,113.96.223.207,23.1167,113.2500,Guangdong,28800.0
4,Beijing,China Unicom Beijing Province Network,Asia,China,111.206.36.15,39.9289,116.3883,Beijing,28800.0
5,Boardman,"Amazon.com, Inc.",North America,United States,52.25.13.124,45.7788,-119.5290,Oregon,-25200.0
6,Beijing,China Unicom Beijing Province Network,Asia,China,123.125.143.15,39.9289,116.3883,Beijing,28800.0
7,Enkoeping,Liden Data Internetwork AB,Europe,Sweden,212.112.33.42,59.6333,17.0667,Uppsala,7200.0
8,NaN,OVH SAS,Europe,France,54.37.85.64,48.8582,2.3387,NaN,7200.0
9,San Francisco,"DigitalOcean, LLC",North America,United States,107.170.252.110,37.7312,-122.3826,California,-25200.0


In [8]:
# create engine
engine = create_engine('sqlite:///apache.sqlite')


In [9]:
# interface with engine via metadata
Base.metadata.create_all(engine)

In [10]:
#start session
from sqlalchemy.orm import Session
session = Session(bind=engine)

In [11]:
#convert dfs to dicts in order to create classes
access = access_logs_df.to_dict(orient='records')
responses = responses_df.to_dict(orient='records')

In [12]:
#create classes
Base = declarative_base()

class Access(Base):
    __tablename__ = 'access'
    
    ip = Column(String, primary_key=True)
    DateTime= Column(String)
    Method = Column(String)
    URL = Column(String)
    ResponseCode = Column(String)
    Referer = Column(String)
    BytesSent = Column(String)
    UserAgent = Column(String)

    def __repr__(self):
        return f"id={self.id}, name={self.access}"

In [13]:
class Responses(Base):
    __tablename__ = 'responses'
    ip = Column(String, primary_key=True)
    city = Column(String)
    connection_isp = Column(String)
    continent_name = Column(String)
    country_name = Column(String)
    latitude = Column(Float)
    longitude = Column(Float)
    region_name = Column(String)
    time_zone_gmt_offset = Column(String)
    
    def __repr__(self):
        return f"id={self.id}, name={self.responses}"

In [14]:
# create db
Base.metadata.create_all(engine)

In [15]:
# check that it worked.
engine.table_names()

['access', 'responses']

In [16]:
# create connection
conn = engine.connect()

In [17]:
# check that all the info is there
Access.__table__

Table('access', MetaData(bind=None), Column('ip', String(), table=<access>, primary_key=True, nullable=False), Column('DateTime', String(), table=<access>), Column('Method', String(), table=<access>), Column('URL', String(), table=<access>), Column('ResponseCode', String(), table=<access>), Column('Referer', String(), table=<access>), Column('BytesSent', String(), table=<access>), Column('UserAgent', String(), table=<access>), schema=None)

In [18]:
Responses.__table__

Table('responses', MetaData(bind=None), Column('ip', String(), table=<responses>, primary_key=True, nullable=False), Column('city', String(), table=<responses>), Column('connection_isp', String(), table=<responses>), Column('continent_name', String(), table=<responses>), Column('country_name', String(), table=<responses>), Column('latitude', Float(), table=<responses>), Column('longitude', Float(), table=<responses>), Column('region_name', String(), table=<responses>), Column('time_zone_gmt_offset', String(), table=<responses>), schema=None)

In [19]:
#delete tables to avoid duplicates when re-run
#conn.execute(Access.__table__.delete())
#conn.execute(Responses.__table__.delete())


In [20]:
# execute connection
conn.execute(Access.__table__.insert(), access)
#conn.execute(Responses.__table__.insert(), responses)


IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: access.ip [SQL: 'INSERT INTO access (ip, "DateTime", "Method", "URL", "ResponseCode", "Referer", "BytesSent", "UserAgent") VALUES (?, ?, ?, ?, ?, ?, ?, ?)'] [parameters: (('62.170.2.205', '2018-04-08 07:19:25', 'HEAD', 'http://198.27.242.168:80/phpmyadmin/', '404', '-', '220', 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'), ('62.170.2.205', '2018-04-08 07:19:25', 'HEAD', 'http://198.27.242.168:80/PMA/', '404', '-', '219', 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'), ('62.170.2.205', '2018-04-08 07:19:25', 'HEAD', 'http://198.27.242.168:80/dbadmin/', '404', '-', '219', 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'), ('62.170.2.205', '2018-04-08 07:19:26', 'HEAD', 'http://198.27.242.168:80/pma/', '404', '-'

In [ ]:
# check that it worked
engine.execute("select ip from access limit 10").fetchall()


In [ ]:
engine.execute('select ip from responses limit 10').fetchall()


In [ ]:
#check what's in there
engine.execute("select * from access limit 1").fetchall()


In [ ]:
engine.execute("select * from responses limit 1").fetchall()